This script demonstrates controlling the robot to grasp both champagne glasses.

### Initialization

In [1]:
'''
This script grasps two champagne glasses, using thumb-index coupling, and middle-ring coupling, respectly.
'''
import numpy as np
import controller_utils
import mujoco
from mujoco import viewer
import tools.rotations as rot
# import rospy
import time
time.sleep(2.0)
# import mediapy as media
import task_planner as tp
from scipy.spatial.transform import Rotation as R

xml_path = 'description/iiwa7_allegro_llm.xml'
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
mujoco.mj_step(model, data)

# view = viewer.launch(model, data)
view = viewer.launch_passive(model, data)
# in notebook, we need ro run view.sync() manually, and set auto_sync=False

obj_names = ['champagne_1', 'champagne_2', 'champagne_3'] # id: 27, 28, 29
r = controller_utils.Robot(model, data, view, auto_sync=False, obj_names=obj_names)

# # home position, palm facing upwards
# q0 = np.array([-0.32032434,  0.02706913, -0.22881953, -1.42621454,  1.3862661 , 0.55966738,  1.79477984 - np.pi*3/2])
# r.d.qpos[:7]  = q0 # robot joint angles
# r.step()
# view.sync()

ee_pose = np.copy(r.x)
hand_joints = np.copy(r.qh) # fully open (no ad/abduction DoFs)
hand_zeros = np.zeros(16)

# to grasp two objects, first rotate the end effector to make the palm facing upwards
# Multiply the quaternions to get the combined rotation
# q_combined = R.from_quat(np.copy(r.x[3:])) * R.from_euler('z', 90, degrees=True)
# ee_home[3:] = q_combined.as_quat() # modify the home pose as the desired one with the palm facing upwards

Kinematic model has been loaded from kinematics/q2pose_left.txt


Move to initial position

In [2]:
goal_pose = np.copy(ee_pose)
goal_pose[1] += 0.3
tp.move_ee(r, view, goal_pose[0], goal_pose[1], goal_pose[2], err_margin=0.001, use_ds=True)

0.30000167606471373
0.29998102891617023
0.29993850068414907
0.299874521092508
0.299789511425317
0.29968388442894
0.29955804419075227
0.2994123860608705
0.2992472966276371
0.29906315375103215
0.2988603266497878
0.29863917603434637
0.29840005427630595
0.298143305604965
0.29786926309564554
0.29757825048250575
0.297270584842888
0.2969465769817935
0.29660653154977096
0.296250747156078
0.2958795164781131
0.29549312636805103
0.29509185795740167
0.2946759867600327
0.29424578277404767
0.2938015105827892
0.2933434294551382
0.29287179344520653
0.2923868514914532
0.29188884751522054
0.29137802051863865
0.29085460468183694
0.290318829459379
0.2897709196758288
0.2892110956203561
0.28863957314028604
0.2880565637335063
0.2874622746396434
0.28685690892993654
0.28624066559634576
0.28561373963301895
0.28497632211761775
0.28432860031806123
0.2836707577757316
0.28300297438207145
0.282325426454223
0.28163828680981756
0.2809417248408094
0.2802359065862829
0.27952099480406717
0.27879714904116926
0.27806452570

## Grasp `champagne_2` on the top of the scene

In [3]:
# (2) Grasp object #3 using middle and ring finger
# move back a bit, then move down, them move to object base
X2, Y2, Z2 = r.x_obj_dict['champagne_2'][:3] # up

goal_pose = np.copy(r.x)
qh0 = np.copy(hand_joints)

goal_pose[0] = X2 - 0.4
goal_pose[1] = Y2 + 0.02
goal_pose[2] = Z2 + 0.03

tp.move_ee(r, view, goal_pose[0], goal_pose[1], goal_pose[2], err_margin=0.001, use_ds=True)
print('Arrived at target position.')

0.15752794554249264
0.1574303944236692
0.1573092683324857
0.15716529220147926
0.1569991333413287
0.15681142396016332
0.15660277132293132
0.15637376224200958
0.15612496517361993
0.15585693127624964
0.15557019502320044
0.1552652746242584
0.15494267236734668
0.15460287492849573
0.15424635367138392
0.1538735649458785
0.15348495038975532
0.15308093723542382
0.1526619386226742
0.15222835391657163
0.15178056903105525
0.15131895675849158
0.15084387710376468
0.15035567762245938
0.14985469376261382
0.1493412492092752
0.14881565623113382
0.14827821602846075
0.14772921908156694
0.14716894549901718
0.1465976653648208
0.14601563908386694
0.14542311772487554
0.14482034336017852
0.14420754940168334
0.14358496093240555
0.1429527950330105
0.14231126110283743
0.14166056117494472
0.14100089022474332
0.14033243647193522
0.139655381675287
0.13896990141972348
0.13827616539619056
0.13757433767368227
0.13686457696321408
0.13614703687379126
0.1354218661602376
0.1346892089628914
0.13394920503915173
0.13320198998

Open hand:

In [4]:
tp.move_hand(r, view, hand_zeros, err_margin=0.5)
print('Hand opened.')

Hand opened.


Move towards object to grasp:

In [5]:
tp.move_ee_relative(r, view, 0.25, 0.05, 0, err_margin=0.01, use_ds=True)

0.2549223973233377
0.25487940007895793
0.25482219122307354
0.25475097165636806
0.2546659375433557
0.2545672805857897
0.2544551882562724
0.2543298440111265
0.2541914274856636
0.25404011467368687
0.2538760780927792
0.2536994869367464
0.2535105072164286
0.2533093018899611
0.2530960309834471
0.2528708517029113
0.2526339185381701
0.25238538335958177
0.25212539539262036
0.25185410127439073
0.25157164525453973
0.25127816936495434
0.25097381347558584
0.25065871534779355
0.2503330106857036
0.24999683318558422
0.24965031458322007
0.24929358469938367
0.24892677148347675
0.24855000105542016
0.24816339774587384
0.24776708413487428
0.24736118108897592
0.24694580779697922
0.24652108180433163
0.24608711904627972
0.24564403387984918
0.24519193911472842
0.2447309460431216
0.24426116446864116
0.243782702734297
0.24329566774964584
0.24280016501714982
0.2422962986577992
0.2417841714360451
0.24126388478408625
0.240735538825551
0.24019923239861452
0.23965506307858198
0.23910312719997553
0.23854351987815184
0

Close fingers to grasp:

In [6]:
F_grasp = [0.5] # notice that the dimension of F_grasp must be identical to the number of pairs provided in the function `r.pinch_grasp_force`

x0 = np.copy(r.x)
qh0 = np.copy(r.qh)

count = 0
while count < 500:
    u_add = r.pinch_grasp_force(F_grasp, pairs=[[1,2]])
    r.moveto_attractor(x0, qh0, scaling=1, u_add=u_add)
    count += 1
    view.sync()
    time.sleep(0.002)

Lift up object:

In [7]:
tp.move_ee_relative(r, view, 0, 0, 0.4, err_margin=0.20, use_ds=True)
print('Grasped object #2.')

0.3999826441276784
0.39994054946807006
0.3998745398670569
0.39978531876174533
0.3996735494060434
0.39953986456945784
0.39938486954471974
0.39920914405655283
0.39901324014404677
0.3987976761492666
0.3985629659374765
0.39830960233322077
0.39803802963763735
0.3977486717389225
0.39744197152009936
0.3971183587433656
0.3967782514327142
0.3964220545919047
0.3960501614455769
0.3956629536908542
0.3952608005004522
0.39484406318688303
0.3944130936019436
0.39396838776549353
0.3935103003321399
0.39303914336231577
0.3925552551371371
0.3920589395360948
0.3915504859288688
0.39103017493377795
0.3904982449390657
0.3899549694837851
0.3894010642248959
0.3888367024897622
0.38826205010407633
0.38767731256645793
0.38708268087846204
0.3864783232178993
0.38586442657227943
0.3852411811080749
0.3846087808631385
0.3839674396821451
0.38331735044622306
0.382658685403538
0.3819916275715153
0.38131639349316104
0.3806331827849076
0.3799421940288289
0.3792436259203455
0.37853767196666444
0.37782454446566627
0.377104422

### Grasp `champagne_3` at the bottom of the scene

Move towards glass #3 and pre-shape:

In [8]:
tp.move_ee_relative(r, view, -0.2, -0.2, 0, err_margin=0.1, use_ds=True)
X3, Y3, Z3 = r.x_obj_dict['champagne_3'][:3] # down

0.2827096235068135
0.2825630320691323
0.28240328672971843
0.28223071373589287
0.2820456172301639
0.28184828964171427
0.28163901296442145
0.2814180593716929
0.2811856821502958
0.2809421420708875
0.2806877224037365
0.2804226285980399
0.2801471734574001
0.2798615866172813
0.2795660810804511
0.27926086003773537
0.2789461168675853
0.27862204642701427
0.27828884090922623
0.2779466891972018
0.2775957767936345
0.27723628585182386
0.27686839522237994
0.27649227877267796
0.2761081104528806
0.27571606700861356
0.275316316989027
0.2749090237960207
0.27449434659798966
0.27407244110790957
0.27364345984670935
0.2732075523805938
0.2727648653954161
0.27231554282269477
0.2718596919964399
0.27139748609101816
0.27092906035362824
0.27045454730459906
0.26997407729173234
0.26948777814478747
0.2689957518240205
0.26849815081809364
0.26799509372161734
0.26748669741485104
0.26697307688712985
0.26645434470306184
0.2659306112113248
0.26540198455458663
0.2648685706949175
0.26433047344600796
0.26378779450861506
0.26

In [9]:
tp.move_ee(r, view, X3-0.3, Y3-0.2, Z3+0.1, err_margin=0.001, use_ds=True)

0.4664466642293161
0.46592364257604213
0.4653828553070846
0.46482486673512635
0.46425022385991027
0.4636594563790221
0.4630530573184709
0.4624315106753014
0.4617952765501361
0.46114475151261836
0.4604804280091875
0.45980272588127713
0.45911205422246437
0.45840880925075456
0.4576933755875769
0.45696612655623836
0.456227424719012
0.4554775767215976
0.45471696773627085
0.4539459358487684
0.45316480073751786
0.45237387725044403
0.45157343199968497
0.45076380540341104
0.4499452919660356
0.44911816565323837
0.44828267637817976
0.4474391270248781
0.4465877523564538
0.4457288567681311
0.44486268369144805
0.4439894821290206
0.4431094916947996
0.4422229154546677
0.4413299836457338
0.4404309515270938
0.4395260336819744
0.43861543882539344
0.4376993806029885
0.43677801504546365
0.43585154257432723
0.4349201557148519
0.43398408784081904
0.43304351136987845
0.4320985496665248
0.4311493672263002
0.43019613255650724
0.42923903583980333
0.4282782329023317
0.4273138748306707
0.42634610904622394
0.425375

Move fingers to pre-grasp position:

In [10]:
tp.move_ee_relative(r, view, 0.25, 0.0, 0.0, err_margin=0.10, use_ds=True)

0.25007711142066413
0.25013977280910304
0.2501881919837355
0.25022257186486513
0.25024311303547686
0.2502500128104114
0.250243465553563
0.2502236626949282
0.25019079275792727
0.2501450413802544
0.25008659135725203
0.2500156226796041
0.2499323125740743
0.2498368355459621
0.2497293634247298
0.24961006540770467
0.24947910810859195
0.24933665559938503
0.24918286945254745
0.24901790878362542
0.248841930293163
0.2486550883085691
0.24845753482530125
0.24824941954751192
0.24803088992529354
0.2478020911922707
0.2475631664021272
0.24731425646408464
0.24705550017740777
0.2467870342650045
0.24650899340621074
0.24622151026306702
0.24592471551680248
0.24561873334633677
0.2453036909454566
0.2449797137133584
0.24464692519134343
0.2443054440033261
0.24395539229383945
0.24359688018009246
0.2432300321799769
0.2428549634255571
0.2424717872249794
0.24208061517729842
0.2416815571752046
0.24127472143150103
0.24086021450592382
0.2404381413366011
0.24000860527366716
0.23957170811466327
0.2391275501410026
0.238

In [11]:
qh = np.copy(r.qh)
qh[:4] = 0
qh[12:] = 0
tp.move_hand(r, view, qh, 0.5)

In [12]:
tp.move_ee_relative(r, view, 0.25, -0.1, 0.05, err_margin=0.27, use_ds=True)

0.27386279954993203
0.2738641859382862
0.2738654651321732
0.27386661538752705
0.27386763555053395
0.2738685621362296
0.27386939722105996
0.2738701296233347
0.2738707313327391
0.273870749742723
0.27387051605996116
0.2738700569122126
0.27386935643428784
0.27386836279861093
0.2738670681689451
0.27386545010838936
0.27386349876836763
0.2738611914028964
0.2738584283575656
0.27385522175763233
0.2738515155948786
0.2738472833187382
0.27384249012821305
0.2738371153390975
0.27383106723259304
0.27382433983783594
0.27381690104502865
0.27380871706794757
0.273799754051714
0.27378997779813874
0.2737793540485915
0.27376793623614964
0.27375557995600447
0.2737422674640853
0.2737279672347226
0.2737126493504588
0.2736962839912062
0.2736788419751382
0.27366028932321984
0.27364059562167925
0.27361973507367127
0.273597647369317
0.273574330064229
0.2735497733670818
0.2735239564156917
0.27349686039222154
0.27346846721501444
0.2734387599306885
0.2734077226658997
0.2733753407015906
0.273341600511426
0.27330648980

In [13]:
# tp.move_ee_relative(r, view, 0.25, 0, 0, err_margin=0.25, use_ds=True)

Close fingers to grasp:

In [14]:
F_grasp = [1.0] # notice that the dimension of F_grasp must be identical to the number of pairs provided in the function `r.pinch_grasp_force`

x0 = np.copy(r.x)
qh0 = np.copy(r.qh)

count = 0
while count < 500:
    u_add = r.pinch_grasp_force(F_grasp, pairs=[[0,3]])
    r.moveto_attractor(x0, qh0, scaling=1, u_add=u_add)
    count += 1
    view.sync()
    time.sleep(0.002)

Lift up object:

In [15]:
tp.move_ee_relative(r, view, 0, 0, 0.4, err_margin=0.28, use_ds=True)
print('Grasped object #2.')

0.3999936412263866
0.399969340779926
0.3999275258566976
0.39986866235535573
0.3997932154256956
0.39970164493129556
0.39959439129580676
0.39947186710589627
0.39933453157976595
0.3991827425340804
0.3990169070369804
0.3988374113163454
0.3986446779255836
0.3984391274265167
0.39822097556861136
0.39799054860534777
0.39774816312863154
0.3974941684114324
0.3972288551883048
0.39695246172504756
0.3966652892426681
0.3963676120422279
0.39605969834472843
0.3957417967904259
0.3954141633061868
0.39507711778702037
0.3947308528126687
0.3943756077908248
0.394011754727
0.3936394173270824
0.39325881882855396
0.3928701283957469
0.3924735641971156
0.39206935115629965
0.391657698921066
0.39123880441142317
0.390812869617164
0.3903802192493341
0.38994102615340903
0.389495462622128
0.38904370235009744
0.38858592214316856
0.38812229047321095
0.3876529804329726
0.387178147221897
0.38669795516795635
0.38621257343272686
0.3857221665324058
0.38522682687625537
0.3847267437538999
0.38422213047054427
0.3837131252089352

Move to trash bin:

In [16]:
tp.move_ee_relative(r, view, -0.4, -0.4, 0, err_margin=0.1, use_ds=True)
tp.move_ee_relative(r, view, 0, 0, -0.3, err_margin=0.1, use_ds=True)

0.5656405859561485
0.5655803562719446
0.5655051463605008
0.5654152561253872
0.5653109932654848
0.5651925933222418
0.5650603242307536
0.564914438716618
0.5647551784035335
0.5645827748102423
0.5643974419207907


0.5641994010011694
0.5639887809457397
0.5637658743952711
0.5635308666647489
0.5632839399472505
0.5630252844626918
0.5627550757079405
0.56247349709452
0.5621807114664167
0.56187685060057
0.5615620805797262
0.5612365535348656
0.5609004787871806
0.5605539945502576
0.5601972377594956
0.5598302262294718
0.5594531808199056
0.5590662880042692
0.5586696292585943
0.5582633335692019
0.557847553290744
0.5574224191167453
0.5569880510037198
0.5565445863963839
0.5560921349991277
0.5556308175039966
0.5551607545748137
0.5546820121190866
0.5541947895763945
0.5536992032704715
0.5531953573199265
0.552683359082805
0.552163316225268
0.5516353371468931
0.5510995317951954
0.5505560160825896
0.5500048888030118
0.5494462665348505
0.5488802407693371
0.5483069393752795
0.5477264396976962
0.5471388459518767
0.5465442526780591
0.545942741988354
0.545334408252319
0.5447193430307534
0.5440976665546573
0.543469465934572
0.542834826025095
0.5421937965141576
0.5415464700068247
0.5408929430355671
0.5402333027043725
0.53

Rotate hand:

In [17]:
# rotate hand
tp.rotate_ee(r, view, 'z', 150, err_margin=0.01)

0.741180528853547
0.740595475772852
0.7394257871464996
0.7376723327296902
0.7353375839701374
0.7324245579150678
0.7289365205107488
0.7248765152420621
0.7202475411165252
0.7151040194446199
0.7095167399653418
0.7035486204450653
0.697255305973743
0.6906858892734664
0.6838837183647386
0.6768870403049441
0.6697296073149016
0.6624411447133691
0.6550479323658285
0.6475731098850922
0.6400370950117877
0.6324578722165857
0.6248512761299782
0.6172312428690674
0.6096100296344038
0.6019983717732541
0.5944057230147207
0.586840428418837
0.5793097444617563
0.5718200174199466
0.564376784686091
0.5569848668388212
0.5496484480724417
0.542371151432806
0.5351561009120267
0.5280059503247179
0.5209229766132368
0.5139091580318015
0.5069661427367873
0.5000952933736114
0.4932977228486788
0.4865743258897086
0.4799258056715221
0.4733526893035771
0.4668553512891014
0.46043403147329753
0.454088847702087
0.4478198091332646
0.4416268287189067
0.4355097325290187
0.42946826989155706
0.42350212214937444
0.41761091047204

In [18]:
tp.rotate_ee(r, view, 'y', 60, err_margin=0.001)

0.29284640225948877
0.29240942475578047
0.2916261059364441
0.2905365278770411
0.2891763978966323
0.2875777502352348
0.285769405644392
0.2837775869480166
0.28162574774914184
0.2793349901805059
0.2769242811475945
0.2744107422062618
0.2718097985254677
0.2691353190555541
0.26639950833598935
0.26361202141084705
0.2607824834840343
0.25792266918171514
0.2550404120691747
0.2521427901324643
0.24923612961470643
0.24631673516983543
0.24338757211936746
0.24045353289754134
0.23751884615164343
0.23458728220959257
0.2316621975622697
0.22874657570433277
0.22584306408621224
0.22295400754271744
0.22008147892676178
0.21722730598484818
0.21439309564825682
0.21158025649927514
0.20879001891102567
0.20602345322571092
0.2032814861205232
0.20056491534651977
0.19787443146814987
0.19521062785628773
0.19257399941841713
0.18996494829799682
0.18738385497325405
0.18483091209737457
0.18230632575072336
0.17981019150817834
0.17734261688857622
0.17490367148643848
0.17249337578376134
0.1701117168698656
0.1677586502518679

Drop objects:

In [19]:
tp.move_hand(r, view, hand_zeros, err_margin=0.5)
print('Hand opened.')

Hand opened.
